## Disaggregation of Monthly Tourism Arrivals to Weekly
#### We aim to disaggregate monthly arrivals to weekly by shaping it with a proxy trend data retrieved from Google and also a bump if there is a public holiday that week

#### 1. Loading and Cleaning the dataset

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

df = pd.read_csv('C:\\Users\\brend\\Downloads\\MASTER DOC DRAFT WORK CSV.csv')

# Clean and prepare the data
# Convert Monthly_Visitor_Arrivals to numeric (remove commas)
df['Monthly_Visitor_Arrivals'] = df['Monthly_Visitor_Arrivals'].replace({',': ''}, regex=True).astype(float)

# Define month order mapping to ensure correct chronological sorting
month_order = {"Jan.": 1, "Feb.": 2, "Mar.": 3, "Apr.": 4, "May": 5, 
               "Jun.": 6, "Jul.": 7, "Aug.": 8, "Sep.": 9, "Oct.": 10, 
               "Nov.": 11, "Dec.": 12}

#### 2. Perform disaggregation for each country separately

In [ ]:
result_dfs = []  # to collect processed DataFrames for each country
for country, cdf in df.groupby('Country/Area'):
    cdf = cdf.copy()

#### 3. Aggregate monthly data for regression (one row per month with summed indicators)

In [ ]:
    monthly_data = cdf.groupby(['Year', 'Month']).agg(
        Monthly_Total=('Monthly_Visitor_Arrivals', 'sum'),
        Holiday_weeks=('Publichol_Top6', 'sum'),        # count of holiday weeks in the month
        Proxy_total=('Proxy_Trend_data', 'sum')         # sum of proxy trend data in the month
    ).reset_index()

# Sort by Year and Month for chronological order
    monthly_data['Month_num'] = monthly_data['Month'].map(month_order)
    monthly_data.sort_values(['Year', 'Month_num'], inplace=True)
    
# Fit OLS regression: Monthly_Total ~ Holiday_weeks + Proxy_total (with intercept)
    if len(monthly_data) >= 3:  # ensure enough data points to fit 3 parameters (intercept, b1, b2)
        X = sm.add_constant(monthly_data[['Holiday_weeks', 'Proxy_total']])
        y = monthly_data['Monthly_Total']
        ols_model = sm.OLS(y, X).fit()
# Regression coefficients
        const, b1, b2 = ols_model.params['const'], ols_model.params['Holiday_weeks'], ols_model.params['Proxy_total']
    else:
# If not enough data points to regress, use a simple fallback (equal or proxy-based distribution)
        const, b1, b2 = 0.0, 0.0, 1.0  # this will use Proxy_Trend_data as the weight

#### 4. Compute weekly predicted values using the regression coefficients

In [ ]:
# Distribute the intercept evenly across 4 weeks (const/4 each week)
    cdf['pred_week'] = (const / 4.0) + (b1 * cdf['Publichol_Top6']) + (b2 * cdf['Proxy_Trend_data'])

#### 5. Scale weekly predictions to match the monthly totals

In [ ]:
# Sum of predictions per month:
    cdf['pred_month_sum'] = cdf.groupby(['Year', 'Month'])['pred_week'].transform('sum')
   
# Monthly total for each group (same value repeated for the group's rows)
    cdf['monthly_total_val'] = cdf.groupby(['Year', 'Month'])['Monthly_Visitor_Arrivals'].transform('sum')
    
# Scale factor = actual total / predicted sum, then multiply each weekly pred
    cdf['scaled_week'] = cdf['pred_week'] * (cdf['monthly_total_val'] / cdf['pred_month_sum'])

#### 6. Handle negative weekly values by setting to 0 and redistributing the difference

In [ ]:
    cdf['adj_week'] = cdf['scaled_week'].copy()
    for (year, month), group in cdf.groupby(['Year', 'Month']):
        weekly_vals = group['adj_week'].values
        if (weekly_vals < 0).any():
            total = group['monthly_total_val'].iloc[0]  # actual monthly total for this group
            neg_sum = -weekly_vals[weekly_vals < 0].sum()  # total amount of negative values (as positive)
# Set negative estimates to 0
            weekly_vals[weekly_vals < 0] = 0.0
# Reduce the other weeks by the proportional amount to keep sum = total
            if weekly_vals.sum() > 0:
                weekly_vals = weekly_vals - (weekly_vals / weekly_vals.sum()) * neg_sum
            else:
# If all weeks were zero (edge case), just split total evenly
                weekly_vals[:] = total / len(weekly_vals)
# Write the adjusted values back
            cdf.loc[group.index, 'adj_week'] = weekly_vals

#### 7. Applying a business rule to enforce holiday week ≥30% more than non-holiday weeks constraint


In [ ]:
    cdf['Weekly_Visitor_Arrivals'] = cdf['adj_week'].copy()  # start from the adjusted values
    for (year, month), group in cdf.groupby(['Year', 'Month']):
        vals = group['Weekly_Visitor_Arrivals'].values
        hol_mask = group['Publichol_Top6'].values.astype(bool)   # True for holiday weeks
        non_mask = ~hol_mask                                     # True for non-holiday weeks
        if hol_mask.any() and non_mask.any():
# Iteratively adjust until holiday condition is met
            for _ in range(10):  # loop safeguard (max 10 iterations)
# Find smallest holiday week and largest non-holiday week
                hol_vals = vals[hol_mask]
                non_vals = vals[non_mask]
                if hol_vals.min() >= 1.3 * non_vals.max():
                    break  # condition satisfied
# Determine required increase for the smallest holiday week
                min_h_idx = hol_mask.nonzero()[0][hol_vals.argmin()]  # index in 'vals' of min holiday week
                required = 1.3 * non_vals.max() - hol_vals.min()
# Increase the smallest holiday week
                vals[min_h_idx] += required
# Decrease non-holiday weeks proportionally to keep the total the same
                non_total = non_vals.sum()
                if non_total > 0:
# Subtract the required amount proportionally from each non-holiday week
                    vals[non_mask] -= (vals[non_mask] / non_total) * required
# If any value went negative due to adjustment, set it to 0 (should not usually happen)
                    vals[vals < 0] = 0.0
# Update the group values in the DataFrame
            cdf.loc[group.index, 'Weekly_Visitor_Arrivals'] = vals
    
# Collect the processed country DataFrame
    result_dfs.append(cdf)

# Concatenate results for all countries
result_df = pd.concat(result_dfs, ignore_index=True)

#### 8. Save the final dataset with updated weekly values to a new CSV file

In [ ]:
result_columns = ['Country/Area', 'Year', 'Month', 'Monthly_Visitor_Arrivals',
                  'Week_Category', 'Publichol_Top6', 'Proxy_Trend_data', 'Weekly_Visitor_Arrivals']
result_df[result_columns].to_csv('C:\\Users\\brend\\Downloads\\MASTER DOC DRAFT WORK CSV_UPDATED.csv', index=False)

print("Disaggregation complete. Results saved to 'weekly_disaggregated_output.csv'.")